# Corporate Hybrid Forecast Notebook ((Prophet vs ARIMA vs TBATS/ETS)) – v3


## 01 - Imports & Settings

In [1]:
"""
Hybrid 3-Way (Prophet vs ARIMA vs TBATS/ETS) with 12-month horizon,
capacity report, and daily capacity plan with language split.
- Stable monthly forecasts (log-scale modeling + safety clipping)
- Temporal reconciliation (monthly -> daily) so sheet2 sums match sheet1
- CV sMAPE always computed (adaptive CV), Best_Model justified
- capacity_error shows historical months (Actual_Volume) from REPORT_START_MONTH

"""

import os
import math
import warnings
import traceback
from typing import Optional, Dict, Tuple

import numpy as np
import pandas as pd

from datetime import datetime

# Forecasting libs
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

try:
    from prophet import Prophet
except Exception:
    Prophet = None

try:
    from tbats import TBATS
except Exception:
    TBATS = None

warnings.filterwarnings("ignore")

# ---------- Configuration ----------
# Inputs (aligned to your real files)
INCOMING_SOURCE_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\input_model\Incoming_new.xlsx"  # Sheet 'Main'
INCOMING_SHEET = "Main"
DEPT_MAP_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\input_model\department.xlsx"
DEPT_MAP_SHEET = "map"
PRODUCTIVITY_PATH = r"C:\Users\pt3canro\Desktop\CAPACITY\input_model\productivity_agents.xlsx"

# Outputs
OUTPUT_XLSX = r"C:\Users\pt3canro\Desktop\CAPACITY\outputs\capacity_forecast_hybrid.xlsx"

# Horizons and switches
H_MONTHS = 12             # monthly forecast horizon
DAILY_HORIZON_DAYS = 90   # daily plan horizon
REPORT_START_MONTH = "2025-01"  # show historical months from this month onward in capacity_error

# Temporal reconciliation: if True, build daily from monthly (top-down)
USE_DAILY_FROM_MONTHLY = True

# Safety clipping against extreme over-forecast
ENABLE_FORECAST_CLIP = True
FORECAST_CLIP_MULTIPLIER = 2.5  # cap to X * rolling 12m mean

# Organization-specific week rule (kept for future use)
WEEKLY_START_THU = True

# Language shares (fixed)
LANGUAGE_SHARES = {
    'English': 0.6435,
    'French': 0.0741,
    'German': 0.0860,
    'Italian': 0.0667,
    'Portuguese': 0.0162,
    'Spanish': 0.1135
}


## 02. Data loading and cleaning

In [2]:
def load_incoming(path: str, sheet_name: Optional[str] = None) -> pd.DataFrame:
    """
    Load daily incoming volumes from Excel/CSV.
    Expected columns: Date, department_id, ticket_total (or construct it).
    """
    if not os.path.exists(path):
        raise FileNotFoundError(
            f"Incoming file not found:\n{path}\n"
            "Please update INCOMING_SOURCE_PATH to the correct location."
        )
    ext = os.path.splitext(path)[1].lower()
    if ext in [".xlsx", ".xlsm", ".xls"]:
        if not sheet_name:
            raise ValueError("Excel file detected but no sheet_name provided (e.g., 'Main').")
        df = pd.read_excel(path, sheet_name=sheet_name, engine="openpyxl")
    elif ext == ".csv":
        df = pd.read_csv(path)
    else:
        raise ValueError(f"Unsupported extension for incoming data: {ext}")

    # Basic columns
    base_required = {'Date', 'department_id'}
    missing_base = base_required - set(df.columns)
    if missing_base:
        raise ValueError(
            f"Incoming file must contain columns: {sorted(list(base_required))}. "
            f"Found columns: {list(df.columns)}. Missing: {sorted(list(missing_base))}"
        )

    # ticket_total creation
    if 'ticket_total' not in df.columns:
        if 'total_incoming' in df.columns:
            df['ticket_total'] = pd.to_numeric(df['total_incoming'], errors='coerce').fillna(0)
        elif {'incoming_from_customers', 'incoming_from_transfers'}.issubset(df.columns):
            df['ticket_total'] = (
                pd.to_numeric(df['incoming_from_customers'], errors='coerce').fillna(0) +
                pd.to_numeric(df['incoming_from_transfers'], errors='coerce').fillna(0)
            )
        else:
            raise ValueError(
                "Incoming file must contain 'ticket_total' or 'total_incoming' or "
                "both 'incoming_from_customers' and 'incoming_from_transfers'. "
                f"Found columns: {list(df.columns)}"
            )

    # Dtypes
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    if df['Date'].isna().any():
        bad = df.loc[df['Date'].isna()]
        raise ValueError(f"Some Date values could not be parsed. Example rows:\n{bad.head(5)}")
    df['department_id'] = df['department_id'].astype(str).str.strip()
    df['ticket_total'] = pd.to_numeric(df['ticket_total'], errors='coerce').fillna(0).astype(float)

    # Optional columns
    if 'department_name' in df.columns:
        df['department_name'] = df['department_name'].astype(str).str.strip()
    else:
        df['department_name'] = None
    if 'vertical' in df.columns:
        df['vertical'] = df['vertical'].astype(str).str.strip()

    return df


def load_dept_map(path: str, sheet: Optional[str] = None) -> pd.DataFrame:
    """
    Load dept mapping -> department_name, vertical.
    """
    if not os.path.exists(path):
        return pd.DataFrame(columns=['department_id', 'department_name', 'vertical'])

    ext = os.path.splitext(path)[1].lower()
    if ext in (".xlsx", ".xlsm", ".xls"):
        if sheet:
            mp = pd.read_excel(path, sheet_name=sheet, engine="openpyxl")
        else:
            xls = pd.ExcelFile(path, engine="openpyxl")
            mp = pd.read_excel(xls, sheet_name=xls.sheet_names[0])
    else:
        mp = pd.read_csv(path)

    rename_map = {
        'dept_id': 'department_id',
        'dept_name': 'department_name',
        'name': 'department_name',
        'segment': 'vertical',
        'vertical_name': 'vertical'
    }
    mp = mp.rename(columns={k: v for k, v in rename_map.items() if k in mp.columns})
    if 'department_id' not in mp.columns:
        raise ValueError(f"Department map must contain 'department_id'. Found: {list(mp.columns)}")

    mp['department_id'] = mp['department_id'].astype(str).str.strip()
    mp['department_name'] = (mp['department_name'].astype(str).str.strip()
                             if 'department_name' in mp.columns else None)
    mp['vertical'] = (mp['vertical'].astype(str).str.strip()
                      if 'vertical' in mp.columns else None)

    return mp[['department_id', 'department_name', 'vertical']].drop_duplicates('department_id')


def load_productivity(path: str) -> pd.DataFrame:
    """
    Load agent productivity and compute dept-level mean tickets per agent-day.
    """
    if not os.path.exists(path):
        raise FileNotFoundError(f"Productivity file not found: {path}")
    df = pd.read_excel(path, engine="openpyxl")
    req = {'Date', 'agent_id', 'department_id', 'prod_total_model'}
    missing = req - set(df.columns)
    if missing:
        raise ValueError(f"productivity_agents.xlsx missing columns: {sorted(list(missing))}. "
                         f"Found: {list(df.columns)}")
    df['Date'] = pd.to_datetime(df['Date'], errors='coerce')
    df['department_id'] = df['department_id'].astype(str).str.strip()
    df['prod_total_model'] = pd.to_numeric(df['prod_total_model'], errors='coerce')

    prod_dept = (df.groupby('department_id', as_index=False)['prod_total_model']
                 .mean()
                 .rename(columns={'prod_total_model': 'avg_tickets_per_agent_day'}))
    return prod_dept


## 03 - Utilities & helpers

In [3]:
def month_floor(dt: pd.Timestamp) -> pd.Timestamp:
    return pd.Timestamp(year=dt.year, month=dt.month, day=1)


def business_days_in_month(year: int, month: int) -> int:
    """Approximate Mon-Fri working days in a month."""
    rng = pd.date_range(start=pd.Timestamp(year=year, month=month, day=1),
                        end=pd.Timestamp(year=year, month=month, day=1) + pd.offsets.MonthEnd(0),
                        freq='D')
    return int(np.sum(rng.weekday < 5))


def smape(y_true, y_pred) -> float:
    """sMAPE robust for intermittent series."""
    y_true = np.array(y_true, dtype=float)
    y_pred = np.array(y_pred, dtype=float)
    denom = (np.abs(y_true) + np.abs(y_pred))
    denom[denom == 0] = 1.0
    return float(np.mean(2.0 * np.abs(y_pred - y_true) / denom) * 100.0)


def apply_mapping(incoming: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    """
    Merge department_name / vertical using department_id.
    Prefer incoming values over mapping values when available.
    """
    merged = incoming.merge(mapping, on='department_id', how='left', suffixes=('', '_map'))

    if 'department_name' not in merged.columns:
        merged['department_name'] = None
    if 'department_name_map' not in merged.columns:
        merged['department_name_map'] = None
    merged['department_name'] = merged['department_name'].fillna(merged['department_name_map']).fillna("Unknown")

    if 'vertical' not in merged.columns:
        merged['vertical'] = None
    if 'vertical_map' not in merged.columns:
        merged['vertical_map'] = None
    merged['vertical'] = merged['vertical'].fillna(merged['vertical_map']).fillna("Unmapped")

    drop_cols = [c for c in merged.columns if c.endswith('_map')]
    merged.drop(columns=drop_cols, inplace=True, errors='ignore')
    return merged

## 04. Modelling (Hybrid 3-Way Prophet / ARIMA / TBATS-ETS))

In [4]:
def _clip_forecast_to_scale(ts_m: pd.Series, fc: pd.Series) -> pd.Series:
    """Optional safety clipping to avoid unreasonable explosions."""
    if not ENABLE_FORECAST_CLIP or ts_m.empty or fc.empty:
        return fc
    # rolling 12-month mean as a soft scale
    r12 = float(ts_m.tail(12).mean()) if len(ts_m) >= 3 else float(ts_m.mean())
    if not np.isfinite(r12) or r12 <= 0:
        return fc.clip(lower=0)
    upper = r12 * FORECAST_CLIP_MULTIPLIER
    return fc.clip(lower=0, upper=upper)


def fit_prophet_monthly_log(ts_m: pd.Series):
    """Fit Prophet on log1p(monthly)."""
    if Prophet is None:
        return None, None
    y = np.log1p(ts_m.values)
    dfp = pd.DataFrame({'ds': ts_m.index.to_timestamp(), 'y': y})
    m = Prophet(weekly_seasonality=False, yearly_seasonality=True, daily_seasonality=False)
    m.fit(dfp)

    def fcast(h_months=H_MONTHS):
        future = m.make_future_dataframe(periods=h_months, freq='MS')
        pred = m.predict(future)
        pred = pred.set_index(pd.PeriodIndex(pred['ds'], freq='M'))['yhat']
        pred = np.expm1(pred.iloc[-h_months:])
        return pd.Series(np.clip(pred, 0, None), index=pred.index)

    return m, fcast


def fit_arima_monthly_log(ts_m: pd.Series):
    """Search SARIMAX on log1p(monthly) with seasonal 12 when length permits."""
    y = np.log1p(ts_m)
    best_aic, best_model = np.inf, None
    best_order, best_seasonal = None, None
    pqs = [0, 1, 2]
    PsQs = [0, 1]
    seasonal = len(ts_m) >= 12
    for p in pqs:
        for d in [1] if len(ts_m) < 24 else [0, 1]:
            for q in pqs:
                for P in ([0, 1] if seasonal else [0]):
                    for D in ([0, 1] if seasonal else [0]):
                        for Q in ([0, 1] if seasonal else [0]):
                            try:
                                model = SARIMAX(y, order=(p, d, q),
                                                seasonal_order=(P, D, Q, 12 if seasonal else 0),
                                                enforce_stationarity=False,
                                                enforce_invertibility=False).fit(disp=False)
                                if model.aic < best_aic:
                                    best_aic = model.aic
                                    best_model = model
                                    best_order = (p, d, q)
                                    best_seasonal = (P, D, Q, 12 if seasonal else 0)
                            except Exception:
                                continue

    def fcast(h_months=H_MONTHS):
        fc_log = best_model.get_forecast(h_months).predicted_mean
        idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
        fc = np.expm1(fc_log)
        s = pd.Series(np.clip(fc, 0, None), index=idx)
        return s

    return best_model, fcast


def fit_tbats_or_ets_monthly_log(ts_m: pd.Series):
    """Fit TBATS if available; else ETS (trend+seasonality) on log1p."""
    y = np.log1p(ts_m)
    if TBATS is not None and len(ts_m) >= 12:
        estimator = TBATS(use_arma_errors=False, seasonal_periods=[12])
        model = estimator.fit(ts_m.to_timestamp())
        def fcast(h_months=H_MONTHS):
            vals_log = model.forecast(steps=h_months)
            idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
            vals = np.expm1(vals_log)
            return pd.Series(np.clip(vals, 0, None), index=idx)
        return model, fcast
    else:
        # ETS on log1p: additive trend + additive seasonality when possible
        seasonal = 12 if len(ts_m) >= 24 else None
        model = ExponentialSmoothing(y, trend='add', seasonal=('add' if seasonal else None),
                                     seasonal_periods=seasonal).fit()
        def fcast(h_months=H_MONTHS):
            vals_log = model.forecast(h_months)
            idx = pd.period_range(ts_m.index[-1] + 1, periods=h_months, freq='M')
            vals = np.expm1(vals_log)
            return pd.Series(np.clip(vals, 0, None), index=idx)
        return model, fcast


def rolling_cv_monthly_adaptive(ts_m: pd.Series) -> Optional[Dict[str, float]]:
    """
    Adaptive rolling-origin CV:
    - if n >= 15 -> h=3
    - if 9 <= n < 15 -> h=1
    Returns mean sMAPE per model on original scale.
    """
    n = len(ts_m)
    if n < 9:
        return None
    h = 3 if n >= 15 else 1
    min_train = max(12, n - (h + 2))  # ensure at least one split
    splits = []
    for start in range(min_train, n - h + 1):
        train = ts_m.iloc[:start]
        test = ts_m.iloc[start:start + h]

        metrics = {}

        # Prophet
        mp, fp = fit_prophet_monthly_log(train)
        if fp is not None:
            try:
                pred = fp(h_months=h)
                metrics['Prophet'] = smape(test.values, pred.values[:h])
            except Exception:
                metrics['Prophet'] = np.inf

        # ARIMA
        try:
            ma, fa = fit_arima_monthly_log(train)
            pred = fa(h_months=h)
            metrics['ARIMA'] = smape(test.values, pred.values[:h])
        except Exception:
            metrics['ARIMA'] = np.inf

        # TBATS/ETS
        try:
            mt, ft = fit_tbats_or_ets_monthly_log(train)
            pred = ft(h_months=h)
            metrics['TBATS/ETS'] = smape(test.values, pred.values[:h])
        except Exception:
            metrics['TBATS/ETS'] = np.inf

        splits.append(metrics)

    dfm = pd.DataFrame(splits)
    return dfm.mean().to_dict()


def select_or_blend_forecasts(fc_dict: Dict[str, pd.Series], cv_scores: Dict[str, float], blend: bool = True):
    """
    Given forecasts per model and CV scores (lower better):
    - select best model or
    - blend with weights ~ 1/sMAPE.
    """
    scores = {k: (v if v is not None and np.isfinite(v) else 1e6) for k, v in cv_scores.items()}
    if not blend:
        best = min(scores, key=scores.get)
        return fc_dict[best], {'winner': best, 'weights': {best: 1.0}}

    inv = {k: (1.0 / v if v > 0 else 0.0) for k, v in scores.items()}
    total = sum(inv.values())
    if total == 0:
        best = min(scores, key=scores.get)
        return fc_dict[best], {'winner': best, 'weights': {best: 1.0}}
    w = {k: inv[k] / total for k in inv}

    # Align indices
    idx = None
    for s in fc_dict.values():
        idx = s.index if idx is None else idx.union(s.index)

    blended = sum(w[k] * fc_dict[k].reindex(idx).fillna(0) for k in fc_dict)
    return blended, {'winner': min(scores, key=scores.get), 'weights': w}


def build_monthly_series(df: pd.DataFrame) -> pd.DataFrame:
    """Aggregate daily incoming to monthly by department."""
    df = df.copy()
    df['month'] = df['Date'].dt.to_period('M')
    monthly = (df.groupby(['department_id', 'month'], as_index=False)['ticket_total']
               .sum()
               .rename(columns={'ticket_total': 'incoming_monthly'}))
    return monthly


def forecast_per_department_monthly(monthly: pd.DataFrame) -> pd.DataFrame:
    """
    Hybrid modelling per department (log-scale) + adaptive CV + optional clipping.
    Returns columns:
    department_id, month, forecast_monthly,
    cv_prophet_smape, cv_arima_smape, cv_tbats_ets_smape,
    winner_model, blend_prophet_w, blend_arima_w, blend_tbats_ets_w
    """
    out_rows = []
    dept_ids = monthly['department_id'].unique().tolist()

    for dept in dept_ids:
        ts = (monthly.loc[monthly['department_id'] == dept, ['month', 'incoming_monthly']]
              .sort_values('month')
              .set_index('month')['incoming_monthly'])
        if not pd.api.types.is_period_dtype(ts.index):
            ts.index = pd.PeriodIndex(ts.index, freq='M')
        n = len(ts)
        if n == 0:
            continue

        cv = {}
        fc_dict: Dict[str, pd.Series] = {}

        # PROPHET
        if Prophet is not None and n >= 12:
            try:
                _, fp = fit_prophet_monthly_log(ts)
                if fp is not None:
                    fc_dict['Prophet'] = fp(H_MONTHS)
            except Exception:
                pass

        # ARIMA
        try:
            _, fa = fit_arima_monthly_log(ts)
            fc_dict['ARIMA'] = fa(H_MONTHS)
        except Exception:
            pass

        # TBATS/ETS
        try:
            _, ft = fit_tbats_or_ets_monthly_log(ts)
            fc_dict['TBATS/ETS'] = ft(H_MONTHS)
        except Exception:
            pass

        # Fallback: naive mean
        if not fc_dict:
            idx = pd.period_range(ts.index[-1] + 1, periods=H_MONTHS, freq='M')
            val = max(0.0, float(ts.mean())) if n > 0 else 0.0
            fc_dict['NaiveMean'] = pd.Series([val] * H_MONTHS, index=idx)

        # CV metrics
        try:
            cv = rolling_cv_monthly_adaptive(ts) or {}
        except Exception:
            cv = {}

        # Blend or select
        if not cv:
            preferred = ['ARIMA', 'TBATS/ETS', 'Prophet', 'NaiveMean']
            winner = next((k for k in preferred if k in fc_dict), list(fc_dict.keys())[0])
            blended = fc_dict[winner]
            meta = {'winner': winner, 'weights': {winner: 1.0}}
        else:
            blended, meta = select_or_blend_forecasts(fc_dict, cv_scores=cv, blend=True)

        # Optional clipping
        blended = _clip_forecast_to_scale(ts, blended)

        for per, val in blended.items():
            out_rows.append({
                'department_id': dept,
                'month': per,
                'forecast_monthly': max(0.0, float(val)),
                'cv_prophet_smape': cv.get('Prophet', np.nan),
                'cv_arima_smape': cv.get('ARIMA', np.nan),
                'cv_tbats_ets_smape': cv.get('TBATS/ETS', np.nan),
                'winner_model': meta['winner'],
                'blend_prophet_w': (meta['weights'].get('Prophet', np.nan) if 'weights' in meta else np.nan),
                'blend_arima_w': (meta['weights'].get('ARIMA', np.nan) if 'weights' in meta else np.nan),
                'blend_tbats_ets_w': (meta['weights'].get('TBATS/ETS', np.nan) if 'weights' in meta else np.nan),
            })

    df_out = pd.DataFrame(out_rows)
    if not df_out.empty:
        df_out['department_id'] = df_out['department_id'].astype(str)
        if not pd.api.types.is_period_dtype(df_out['month']):
            df_out['month'] = pd.PeriodIndex(df_out['month'], freq='M')
    return df_out


def compute_monthly_accuracy_with_history(monthly: pd.DataFrame,
                                          fc_monthly: pd.DataFrame,
                                          report_start: str) -> pd.DataFrame:
    """
    Build capacity_error-like table:
    - historical months from report_start .. last_actual: Actual_Volume (Forecast = NaN)
    - future months: Forecast (Actual_Volume = NaN)
    """
    monthly = monthly.copy()
    monthly['department_id'] = monthly['department_id'].astype(str)
    if not pd.api.types.is_period_dtype(monthly['month']):
        monthly['month'] = pd.PeriodIndex(monthly['month'], freq='M')

    fc = fc_monthly.copy()
    fc['department_id'] = fc['department_id'].astype(str)
    if not pd.api.types.is_period_dtype(fc['month']):
        fc['month'] = pd.PeriodIndex(fc['month'], freq='M')

    # Define reporting range
    start_per = pd.Period(report_start, freq='M')
    last_actual = monthly['month'].max()

    # Historical frame
    hist = (monthly.loc[monthly['month'] >= start_per, ['department_id', 'month', 'incoming_monthly']]
            .rename(columns={'incoming_monthly': 'Actual_Volume'}))
    hist['Forecast'] = np.nan

    # Future frame
    fut = fc[['department_id', 'month', 'forecast_monthly',
              'cv_prophet_smape', 'cv_arima_smape', 'cv_tbats_ets_smape',
              'winner_model', 'blend_prophet_w', 'blend_arima_w', 'blend_tbats_ets_w']].copy()
    fut = fut.loc[fut['month'] > last_actual]
    fut = fut.rename(columns={'forecast_monthly': 'Forecast'})
    fut['Actual_Volume'] = np.nan

    # Union
    base = pd.concat([hist, fut], ignore_index=True, sort=False)

    # Accuracy only when both available (mostly future months will be NaN)
    base['Forecast_Accuracy'] = np.where(
        (base['Actual_Volume'].notna()) & (base['Forecast'].notna()) & (base['Actual_Volume'] > 0),
        (1 - (np.abs(base['Forecast'] - base['Actual_Volume']) / base['Actual_Volume'])) * 100.0,
        np.nan
    )
    return base


def compute_capacity_monthly(cap_df: pd.DataFrame, prod_dept: pd.DataFrame) -> pd.DataFrame:
    """Compute FTE/day needed per month = Forecast / (avg_tickets_per_agent_day * workdays_in_month)."""
    out = cap_df.merge(prod_dept, on='department_id', how='left')
    out['avg_tickets_per_agent_day'] = out['avg_tickets_per_agent_day'].replace(0, np.nan)
    out['workdays_in_month'] = [business_days_in_month(m.start_time.year, m.start_time.month) for m in out['month']]
    out['Capacity_FTE_per_day'] = np.where(
        (out['avg_tickets_per_agent_day'] > 0) & (out['workdays_in_month'] > 0) & (out['Forecast'].notna()),
        out['Forecast'] / (out['avg_tickets_per_agent_day'] * out['workdays_in_month']),
        np.nan
    )
    return out


def build_cv_table(fc_monthly: pd.DataFrame, mapping: pd.DataFrame) -> pd.DataFrame:
    """Build mape_table_cv with sMAPE, best model and weights per department."""
    if fc_monthly is None or fc_monthly.empty:
        raise ValueError("fc_monthly is empty; cannot build CV table.")
    cols_keep = [
        'department_id',
        'cv_prophet_smape', 'cv_arima_smape', 'cv_tbats_ets_smape',
        'winner_model',
        'blend_prophet_w', 'blend_arima_w', 'blend_tbats_ets_w'
    ]
    df = (fc_monthly[cols_keep]
          .drop_duplicates(subset=['department_id'])
          .copy())
    df = df.rename(columns={
        'cv_prophet_smape': 'sMAPE_Prophet_CV',
        'cv_arima_smape': 'sMAPE_ARIMA_CV',
        'cv_tbats_ets_smape': 'sMAPE_TBATS_ETS_CV',
        'winner_model': 'Best_Model',
        'blend_prophet_w': 'Weight_Prophet',
        'blend_arima_w': 'Weight_ARIMA',
        'blend_tbats_ets_w': 'Weight_TBATS_ETS',
    })
    df['department_id'] = df['department_id'].astype(str)
    df = apply_mapping(df, mapping)
    ordered_cols = [
        'department_id', 'department_name', 'vertical',
        'sMAPE_Prophet_CV', 'sMAPE_ARIMA_CV', 'sMAPE_TBATS_ETS_CV',
        'Best_Model',
        'Weight_Prophet', 'Weight_ARIMA', 'Weight_TBATS_ETS'
    ]
    df = df[ordered_cols]
    return df.sort_values(['vertical', 'department_id'])


## 05. Daily plan (reconciled)

In [5]:
def dow_profile(g: pd.DataFrame) -> pd.Series:
    """Build normalized day-of-week profile for a department, fallback to uniform."""
    prof = (g.assign(dow=g['Date'].dt.dayofweek)
              .groupby('dow')['ticket_total']
              .mean())
    if prof.notna().sum() >= 3:
        prof = prof / prof.mean()
    else:
        prof = pd.Series(1.0, index=range(7))
    return prof


def disaggregate_month_to_days(dept_df: pd.DataFrame,
                               month_period: pd.Period,
                               target_sum: float) -> pd.DataFrame:
    """
    Allocate monthly forecast to each day in that month using recent DOW profile,
    guaranteeing that daily sum equals the monthly target.
    """
    # Date range for the target month
    start = month_period.start_time
    end = month_period.end_time
    days = pd.date_range(start=start, end=end, freq='D')

    # Build recent DOW profile from last 90 actual days
    hist = dept_df.sort_values('Date').tail(90)
    profile = dow_profile(hist)

    weights = np.array([profile.get(d.dayofweek, 1.0) for d in days], dtype=float)
    weights = np.maximum(weights, 1e-6)
    weights = weights / weights.sum()

    alloc = target_sum * weights
    return pd.DataFrame({'Date': days, 'forecast_daily': alloc})


def build_daily_from_monthly(incoming: pd.DataFrame,
                             fc_monthly: pd.DataFrame,
                             horizon_days: int) -> pd.DataFrame:
    """
    Top-down daily plan:
    - For each department and future month within horizon window,
      disaggregate monthly forecast into daily using DOW profile.
    """
    last_date = incoming['Date'].max()
    start = last_date + pd.Timedelta(days=1)
    end = start + pd.Timedelta(days=horizon_days - 1)
    future_months = pd.period_range(start=start.to_period('M'),
                                    end=end.to_period('M'), freq='M')

    rows = []
    for dept, g in incoming.groupby('department_id'):
        for m in future_months:
            fcm = fc_monthly[(fc_monthly['department_id'] == dept) & (fc_monthly['month'] == m)]
            if fcm.empty:
                continue
            target = float(fcm['forecast_monthly'].iloc[0])
            if target <= 0:
                continue
            alloc_df = disaggregate_month_to_days(g, m, target)
            # Keep only days inside the [start, end] window
            alloc_df = alloc_df[(alloc_df['Date'] >= start) & (alloc_df['Date'] <= end)]
            alloc_df.insert(0, 'department_id', dept)
            rows.append(alloc_df)

    df = pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=['department_id', 'Date', 'forecast_daily'])
    # Fill any missing dept-days (rare) with a small epsilon 0
    return df


def split_daily_by_language(df_daily_fc: pd.DataFrame) -> pd.DataFrame:
    """Split daily forecast by fixed language shares."""
    parts = []
    for lang, w in LANGUAGE_SHARES.items():
        tmp = df_daily_fc.copy()
        tmp['language'] = lang
        tmp['forecast_daily_language'] = tmp['forecast_daily'] * w
        parts.append(tmp)
    out = pd.concat(parts, ignore_index=True)
    return out


def build_daily_capacity_plan(incoming: pd.DataFrame,
                              mapping: pd.DataFrame,
                              prod_dept: pd.DataFrame,
                              fc_monthly: pd.DataFrame,
                              horizon_days: int) -> pd.DataFrame:
    """
    End-to-end daily plan:
    - If USE_DAILY_FROM_MONTHLY: disaggregate monthly forecast (reconciled)
    - Else: robust daily baseline (28D moving average)
    - Split by languages
    - Attach department_name / vertical
    - Compute FTE per day per department/language
    """
    if USE_DAILY_FROM_MONTHLY:
        daily_fc = build_daily_from_monthly(incoming, fc_monthly, horizon_days)
    else:
        daily_fc = forecast_daily_baseline(incoming, horizon_days)

    daily_fc_lang = split_daily_by_language(daily_fc)

    # Attach names/vertical
    daily_fc_lang = apply_mapping(daily_fc_lang, mapping)

    # Merge productivity
    daily_fc_lang = daily_fc_lang.merge(prod_dept, on='department_id', how='left')

    # Compute FTE requirement per day
    daily_fc_lang['avg_tickets_per_agent_day'] = pd.to_numeric(daily_fc_lang['avg_tickets_per_agent_day'], errors='coerce')
    daily_fc_lang['FTE_per_day'] = np.where(
        daily_fc_lang['avg_tickets_per_agent_day'] > 0,
        daily_fc_lang['forecast_daily_language'] / daily_fc_lang['avg_tickets_per_agent_day'],
        np.nan
    )
    cols = ['Date', 'department_id', 'department_name', 'vertical', 'language',
            'forecast_daily_language', 'FTE_per_day']
    daily_plan = daily_fc_lang[cols].sort_values(['Date', 'vertical', 'department_id', 'language'])
    return daily_plan


def forecast_daily_baseline(df_daily: pd.DataFrame, horizon_days: int) -> pd.DataFrame:
    """
    Independent daily baseline (kept as optional):
    - If >=28 days: 28-day moving average per department with DOW profile
    - Else: historical average
    """
    df = df_daily.copy()
    df['Date'] = pd.to_datetime(df['Date'])
    df['department_id'] = df['department_id'].astype(str).str.strip()
    df = df.sort_values(['department_id', 'Date'])
    last_date = df['Date'].max()
    if pd.isna(last_date):
        raise ValueError("forecast_daily_baseline: No valid dates in incoming.")
    start = last_date + pd.Timedelta(days=1)
    idx_future = pd.date_range(start=start, periods=horizon_days, freq='D')

    rows = []
    for dept, g in df.groupby('department_id'):
        g = g.sort_values('Date')
        if len(g) >= 28:
            roll_mean = (g.set_index('Date')['ticket_total']
                         .rolling(window=28, min_periods=1)
                         .mean()
                         .iloc[-1])
            base = float(roll_mean) if np.isfinite(roll_mean) else float(g['ticket_total'].mean())
        else:
            base = float(g['ticket_total'].mean())

        prof = dow_profile(g)
        vals = []
        for d in idx_future:
            w = prof[d.dayofweek] if d.dayofweek in prof.index else 1.0
            vals.append(max(0.0, base * float(w)))
        rows.append(pd.DataFrame({'department_id': dept, 'Date': idx_future, 'forecast_daily': vals}))

    return pd.concat(rows, ignore_index=True) if rows else pd.DataFrame(columns=['department_id', 'Date', 'forecast_daily'])


## 06. Main entry point and excel writing

In [6]:
def main():
    # 1) Load inputs
    incoming = load_incoming(INCOMING_SOURCE_PATH, sheet_name=INCOMING_SHEET)
    mapping = load_dept_map(DEPT_MAP_PATH, DEPT_MAP_SHEET)
    prod = load_productivity(PRODUCTIVITY_PATH)

    # 2) Monthly forecast
    monthly = build_monthly_series(incoming)
    fc_monthly = forecast_per_department_monthly(monthly)

    # 3) capacity_error (historicals + future forecast)
    cap_err = compute_monthly_accuracy_with_history(monthly, fc_monthly, REPORT_START_MONTH)
    cap_err = compute_capacity_monthly(cap_err, prod)
    cap_err = apply_mapping(cap_err, mapping)

    # 4) Daily plan (reconciled with monthly by default)
    daily_capacity_plan = build_daily_capacity_plan(incoming, mapping, prod, fc_monthly, DAILY_HORIZON_DAYS)

    # 5) CV table
    cv_table = build_cv_table(fc_monthly, mapping)

    # 6) Write Excel with required sheet names
    with pd.ExcelWriter(OUTPUT_XLSX, engine="openpyxl", mode="w") as w:
        # Rename monthly sheet exactly as requested
        cap_err[['vertical', 'department_id', 'department_name', 'month',
                 'Actual_Volume', 'Forecast', 'Forecast_Accuracy',
                 'Capacity_FTE_per_day',
                 'winner_model', 'cv_prophet_smape', 'cv_arima_smape', 'cv_tbats_ets_smape',
                 'blend_prophet_w', 'blend_arima_w', 'blend_tbats_ets_w']]\
            .sort_values(['vertical', 'department_id', 'month']).to_excel(w, "capacity_error", index=False)

        daily_capacity_plan.to_excel(w, "daily_capacity_plan", index=False)
        cv_table.to_excel(w, "mape_table_cv", index=False)

    print("Excel written:", OUTPUT_XLSX)


if __name__ == "__main__":
    main()

13:05:17 - cmdstanpy - INFO - Chain [1] start processing
13:05:19 - cmdstanpy - INFO - Chain [1] done processing
13:05:38 - cmdstanpy - INFO - Chain [1] start processing
13:05:39 - cmdstanpy - INFO - Chain [1] done processing
13:06:43 - cmdstanpy - INFO - Chain [1] start processing
13:06:49 - cmdstanpy - INFO - Chain [1] done processing
13:07:05 - cmdstanpy - INFO - Chain [1] start processing
13:07:12 - cmdstanpy - INFO - Chain [1] done processing
13:07:34 - cmdstanpy - INFO - Chain [1] start processing
13:07:34 - cmdstanpy - INFO - Chain [1] done processing
13:07:53 - cmdstanpy - INFO - Chain [1] start processing
13:07:53 - cmdstanpy - INFO - Chain [1] done processing
13:08:16 - cmdstanpy - INFO - Chain [1] start processing
13:08:19 - cmdstanpy - INFO - Chain [1] done processing
13:08:42 - cmdstanpy - INFO - Chain [1] start processing
13:08:43 - cmdstanpy - INFO - Chain [1] done processing
13:10:29 - cmdstanpy - INFO - Chain [1] start processing
13:10:30 - cmdstanpy - INFO - Chain [1]

Excel written: C:\Users\pt3canro\Desktop\CAPACITY\outputs\capacity_forecast_hybrid.xlsx
